In [38]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import os
from PyPDF2 import PdfReader
import numpy as np

In [ ]:
df_1 = pd.read_excel('truoc_ma_so_1802025.xlsx', skiprows=0).iloc[:,:3].fillna('')
df_2 = pd.read_excel('phan_loai_1802025.xlsx', skiprows=0).iloc[:,:3].fillna('')

df = pd.concat([df_1,df_2])
df.columns = ['link','doc_no','doc_date']
df = df.loc[df["doc_no"] != "lỗi"]
df = df.drop_duplicates()
df['doc_name'] = df['doc_no'].replace('/', '_', regex=True).replace('-', '_', regex=True)+'_'+df['doc_date'].replace('-', '_', regex=True)


In [26]:
df.to_excel("link_for_download_pdf_18022025.xlsx",index = True)

In [27]:
# Đường dẫn để đăng nhập
login_url = 'https://thuvienphapluat.vn/page/ajaxcontroler.aspx'

# Dữ liệu cần gửi để đăng nhập
payload = {
    'l_txtUser': 'UNICUSVN',
    'l_txtPass': 'UNIUNI',
    'action': 'Login'
}

# Headers cần thiết cho yêu cầu
headers = {
    'Accept': '*/*',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'no-cache',
    'Content-Type': 'application/x-www-form-urlencoded',
    'DNT': '1',
    'Origin': 'https://thuvienphapluat.vn',
    'Pragma': 'no-cache',
    'Referer': 'https://thuvienphapluat.vn/page/login.aspx?',
    'Sec-CH-UA': '"Chromium";v="130", "Google Chrome";v="130", "Not?A_Brand";v="99"',
    'Sec-CH-UA-Mobile': '?0',
    'Sec-CH-UA-Platform': '"Windows"',
    'Sec-Fetch-Dest': 'empty',
    'Sec-Fetch-Mode': 'cors',
    'Sec-Fetch-Site': 'same-origin',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest'
}

# Khởi tạo session để giữ cookie
session = requests.Session()

# Gửi yêu cầu POST để đăng nhập
login_response = session.post(login_url, data=payload, headers=headers)
# Kiểm tra đăng nhập thành công
if login_response.ok:
    print("Đăng nhập thành công!")
else:
    print("Đăng nhập thất bại:", login_response.status_code)


Đăng nhập thành công!


In [28]:
def get_downloadlink(link):
    response = session.get(link)
    soup = BeautifulSoup(response.text, 'html.parser')
    try:
        find_pdf_link = soup.find('div',id = 'tab8').find('a',id ='ctl00_Content_ThongTinVB_pdfHyperLink')
        if find_pdf_link:
            all_script = soup.find_all('script')
            pattern = r'/documents/download\.aspx\?id=[\w%]+'


            for idx,text in enumerate(all_script):
                if '__urldl' in text.get_text():
                    matches = re.findall(pattern, text.get_text())
                    if matches:
                        print(matches)

            matches[-1]
            download_url = f'https://thuvienphapluat.vn{matches[-1]}&part=-0'
            print(download_url)
            return download_url
        else:
            print("không có link pdf")
    except  Exception as e:
        print(e)

In [29]:
def download_pdf(download_link,file_name,idx):
    path = r"D:\\Onedrive\\OneDrive - UNI CONSULTING\\Eximhow_pdf\\18022025\\"
    # Gửi yêu cầu tải file PDF
    response = session.get(download_link, headers=headers, stream=True)

    # Kiểm tra nếu yêu cầu thành công và lưu file
    if response.ok:
        
        with open(f"{path}{file_name}_{idx}.pdf", "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)
        print(f"Tải file thành công và lưu thành {file_name}.pdf")
    else:
        print("Lỗi khi tải file:", response.status_code)
        


In [30]:
df.iloc[20,0]

'https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-6331-TB-TCHQ-2024-ket-qua-xac-dinh-truoc-ma-so-FLUX-TLF-800M-1-6-638902.aspx'

In [31]:
get_downloadlink(df.iloc[20,0])

['/documents/download.aspx?id=xm03heuqAlTrDAcYznWPiA%3d%3d']
https://thuvienphapluat.vn/documents/download.aspx?id=xm03heuqAlTrDAcYznWPiA%3d%3d&part=-0


'https://thuvienphapluat.vn/documents/download.aspx?id=xm03heuqAlTrDAcYznWPiA%3d%3d&part=-0'

In [32]:
df

,link,doc_no,doc_date,doc_name
0,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,6450/TB-TCHQ,23-12-2024,6450_TB_TCHQ_23_12_2024
2,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,5951/TB-TCHQ,29-11-2024,5951_TB_TCHQ_29_11_2024
3,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,5317/TB-TCHQ,31-10-2024,5317_TB_TCHQ_31_10_2024
4,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,284/TB-TCHQ,17-01-2025,284_TB_TCHQ_17_01_2025
5,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,6058/TB-TCHQ,04-12-2024,6058_TB_TCHQ_04_12_2024
...,...,...,...,...
86,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,6208/TB-TCHQ,11-12-2024,6208_TB_TCHQ_11_12_2024
87,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,301/TB-TCHQ,17-01-2025,301_TB_TCHQ_17_01_2025
88,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,6011/TB-TCHQ,02-12-2024,6011_TB_TCHQ_02_12_2024
89,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,6440/TB-TCHQ,23-12-2024,6440_TB_TCHQ_23_12_2024


In [33]:
for idx,row in df.iloc[72:,:].iterrows():
    print (idx)
    print(row[0])
    download_link= get_downloadlink(row[0])
    print(row[3])
    file_name = row[3]
    try:
        download_pdf(download_link,file_name,idx)
    except Exception as e:
        print(e)

77
https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-6317-TB-TCHQ-2024-ket-qua-xac-dinh-truoc-ma-so-doi-voi-Li2-L-Eyestay-Dial-A-640017.aspx


C:\Users\nguye\AppData\Local\Temp\ipykernel_26984\1446626439.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(row[0])
C:\Users\nguye\AppData\Local\Temp\ipykernel_26984\1446626439.py:4: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  download_link= get_downloadlink(row[0])


['/documents/download.aspx?id=63txNnQgCKyNNsZgqlHcwQ%3d%3d']
https://thuvienphapluat.vn/documents/download.aspx?id=63txNnQgCKyNNsZgqlHcwQ%3d%3d&part=-0
6317_TB_TCHQ_17_12_2024


C:\Users\nguye\AppData\Local\Temp\ipykernel_26984\1446626439.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print(row[3])
C:\Users\nguye\AppData\Local\Temp\ipykernel_26984\1446626439.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  file_name = row[3]


Tải file thành công và lưu thành 6317_TB_TCHQ_17_12_2024.pdf
78
https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-6316-TB-TCHQ-2024-ket-qua-xac-dinh-truoc-ma-so-doi-voi-H-i1-PowerPull-L-Dial-A-640016.aspx
['/documents/download.aspx?id=TbAnntxbgmajoOsexT2QHw%3d%3d']
https://thuvienphapluat.vn/documents/download.aspx?id=TbAnntxbgmajoOsexT2QHw%3d%3d&part=-0
6316_TB_TCHQ_17_12_2024
Tải file thành công và lưu thành 6316_TB_TCHQ_17_12_2024.pdf
79
https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-19-TB-TCHQ-2025-ket-qua-xac-dinh-truoc-ma-so-doi-voi-Lace-Guide-Web-15mm-Soft-Nylon-639597.aspx
['/documents/download.aspx?id=%2bT%2fLr8K7etkfRRVcfKieQA%3d%3d']
https://thuvienphapluat.vn/documents/download.aspx?id=%2bT%2fLr8K7etkfRRVcfKieQA%3d%3d&part=-0
19_TB_TCHQ_02_01_2025
Tải file thành công và lưu thành 19_TB_TCHQ_02_01_2025.pdf
80
https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-6207-TB-TCHQ-2024-ket-qua-xac-dinh-truoc-ma-so-doi-voi-Hyalo4-Skin-Gel-635578.aspx
['

In [35]:
df.to_excel("link_pdf_file_mapping.xlsx",index = True)

In [34]:
get_downloadlink('https://thuvienphapluat.vn/van-ban/Xuat-nhap-khau/Thong-bao-5432-TB-TCHQ-2024-ket-qua-xac-dinh-truoc-ma-so-doi-voi-LEAD-FREE-FLUX-631993.aspx')

['/documents/download.aspx?id=ihtxmL0S0%2bBL%2bDfHGyd%2bjg%3d%3d']
https://thuvienphapluat.vn/documents/download.aspx?id=ihtxmL0S0%2bBL%2bDfHGyd%2bjg%3d%3d&part=-0


'https://thuvienphapluat.vn/documents/download.aspx?id=ihtxmL0S0%2bBL%2bDfHGyd%2bjg%3d%3d&part=-0'

In [39]:
links = []
err_links = []
def is_pdf_valid(file_path):
    try:
        # Mở và kiểm tra file PDF
        with open(file_path, "rb") as f:
            pdf = PdfReader(f)
            pdf.pages[0]  # Kiểm tra trang đầu tiên để xác nhận file không lỗi
        return True  # File PDF hợp lệ
    except Exception as e:
        print(f"File PDF {file_path} có lỗi: {e}")
        return False  # File PDF không hợp lệ

def clean_invalid_pdfs(folder_path):
    for file_name in os.listdir(folder_path):
        # Chỉ kiểm tra file có phần mở rộng .pdf
        if file_name.lower().endswith('.pdf'):
            file_path = os.path.join(folder_path, file_name)
            
            # Kiểm tra tính hợp lệ của file PDF
            if not is_pdf_valid(file_path):
                err_links.append(file_path)
                print(f"Xóa file bị lỗi: {file_path}")
                os.remove(file_path)  # Xóa file nếu không hợp lệ
            else: 
                link = {}
                link["file_name"] = file_name
                link["file_name_2"] = re.sub(r'_\d+\.pdf', '', file_name)
                links.append(link)
                
# Đường dẫn tới thư mục chứa các file PDF
download_folder = r"D:\\Onedrive\\OneDrive - UNI CONSULTING\\Eximhow_pdf\\18022025\\"
clean_invalid_pdfs(download_folder)


In [40]:
download_link_df= pd.DataFrame(links,columns=["file_name","file_name_2"])

In [41]:
download_link_df

,file_name,file_name_2
0,119_TB_TCHQ_08_01_2025_47.pdf,119_TB_TCHQ_08_01_2025
1,123_TB_TCHQ_08_01_2025_10.pdf,123_TB_TCHQ_08_01_2025
2,185_TB_TCHQ_10_01_2025_33.pdf,185_TB_TCHQ_10_01_2025
3,19_TB_TCHQ_02_01_2025_79.pdf,19_TB_TCHQ_02_01_2025
4,20_TB_TCHQ_02_01_2025_18.pdf,20_TB_TCHQ_02_01_2025
5,5282_TB_TCHQ_30_10_2024_49.pdf,5282_TB_TCHQ_30_10_2024
6,5317_TB_TCHQ_31_10_2024_3.pdf,5317_TB_TCHQ_31_10_2024
7,5432_TB_TCHQ_06_11_2024_90.pdf,5432_TB_TCHQ_06_11_2024
8,5433_TB_TCHQ_06_11_2024_35.pdf,5433_TB_TCHQ_06_11_2024
9,5510_TB_TCHQ_08_11_2024_62.pdf,5510_TB_TCHQ_08_11_2024


In [42]:
df

,link,doc_no,doc_date,doc_name
0,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,6450/TB-TCHQ,23-12-2024,6450_TB_TCHQ_23_12_2024
2,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,5951/TB-TCHQ,29-11-2024,5951_TB_TCHQ_29_11_2024
3,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,5317/TB-TCHQ,31-10-2024,5317_TB_TCHQ_31_10_2024
4,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,284/TB-TCHQ,17-01-2025,284_TB_TCHQ_17_01_2025
5,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,6058/TB-TCHQ,04-12-2024,6058_TB_TCHQ_04_12_2024
...,...,...,...,...
86,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,6208/TB-TCHQ,11-12-2024,6208_TB_TCHQ_11_12_2024
87,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,301/TB-TCHQ,17-01-2025,301_TB_TCHQ_17_01_2025
88,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,6011/TB-TCHQ,02-12-2024,6011_TB_TCHQ_02_12_2024
89,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,6440/TB-TCHQ,23-12-2024,6440_TB_TCHQ_23_12_2024


In [43]:
df_copy = df.merge(download_link_df, left_on='doc_name', right_on='file_name_2', how='left')

In [44]:
df_copy

,link,doc_no,doc_date,doc_name,file_name,file_name_2
0,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,6450/TB-TCHQ,23-12-2024,6450_TB_TCHQ_23_12_2024,6450_TB_TCHQ_23_12_2024_0.pdf,6450_TB_TCHQ_23_12_2024
1,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,5951/TB-TCHQ,29-11-2024,5951_TB_TCHQ_29_11_2024,5951_TB_TCHQ_29_11_2024_2.pdf,5951_TB_TCHQ_29_11_2024
2,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,5317/TB-TCHQ,31-10-2024,5317_TB_TCHQ_31_10_2024,5317_TB_TCHQ_31_10_2024_3.pdf,5317_TB_TCHQ_31_10_2024
3,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,284/TB-TCHQ,17-01-2025,284_TB_TCHQ_17_01_2025,NaN,NaN
4,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,6058/TB-TCHQ,04-12-2024,6058_TB_TCHQ_04_12_2024,NaN,NaN
...,...,...,...,...,...,...
81,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,6208/TB-TCHQ,11-12-2024,6208_TB_TCHQ_11_12_2024,6208_TB_TCHQ_11_12_2024_86.pdf,6208_TB_TCHQ_11_12_2024
82,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,301/TB-TCHQ,17-01-2025,301_TB_TCHQ_17_01_2025,NaN,NaN
83,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,6011/TB-TCHQ,02-12-2024,6011_TB_TCHQ_02_12_2024,6011_TB_TCHQ_02_12_2024_88.pdf,6011_TB_TCHQ_02_12_2024
84,https://thuvienphapluat.vn/van-ban/Xuat-nhap-k...,6440/TB-TCHQ,23-12-2024,6440_TB_TCHQ_23_12_2024,6440_TB_TCHQ_23_12_2024_89.pdf,6440_TB_TCHQ_23_12_2024


In [45]:
df_copy.to_excel("downloaded_links_18022025.xlsx", index=False)
